In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

import sys
sys.path.append('../.')
from lib import get_data

../lib


In [2]:
df = get_data.get_model_data(date_range=(0,14), pred_day=21)
df.head()

,state,county,fips,cases,deaths,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,dewPtAvg,...,retail_and_recreation_percent_change_from_baseline,residential_percent_change_from_baseline,workplaces_percent_change_from_baseline,transit_stations_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,income_2018,pop_2018,day_21_delta_cases,day_21_delta_deaths
0,alabama,baldwin,1003,34.266667,0.733333,0.000000,30.800000,89.066667,50.306667,56.046667,...,-43.533333,13.066667,-34.333333,-28.666667,-14.400000,-36.133333,45596.0,218022.0,43,1
1,alabama,chambers,1017,57.466667,3.800000,0.000000,29.044444,87.155556,47.024444,52.260000,...,-28.800000,12.800000,-38.400000,-28.500000,2.153846,NaN,33859.0,33615.0,87,2
2,alabama,elmore,1051,17.133333,0.000000,0.333333,33.933333,90.133333,47.853333,53.253333,...,-31.000000,15.166667,-37.533333,NaN,0.133333,27.571429,42269.0,81887.0,25,0
3,alabama,jefferson,1073,94.000000,0.000000,3.866667,48.155556,92.511111,53.826667,59.091111,...,-34.933333,13.066667,-32.533333,-28.133333,2.533333,13.133333,54730.0,659300.0,172,9
4,alabama,lauderdale,1077,15.000000,1.733333,0.066667,36.733333,90.866667,44.940000,49.646667,...,-36.733333,13.142857,-35.866667,NaN,-5.400000,NaN,37151.0,92387.0,3,1


In [3]:
df = df._get_numeric_data().drop(['fips'],axis=1).dropna()
df.head()

,cases,deaths,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,dewPtAvg,dewPtMax,feelsLikeMin,feelsLikeAvg,...,retail_and_recreation_percent_change_from_baseline,residential_percent_change_from_baseline,workplaces_percent_change_from_baseline,transit_stations_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,income_2018,pop_2018,day_21_delta_cases,day_21_delta_deaths
0,34.266667,0.733333,0.000000,30.800000,89.066667,50.306667,56.046667,62.120000,57.486667,69.293333,...,-43.533333,13.066667,-34.333333,-28.666667,-14.400000,-36.133333,45596.0,218022.0,43,1
3,94.000000,0.000000,3.866667,48.155556,92.511111,53.826667,59.091111,64.835556,57.366667,67.015556,...,-34.933333,13.066667,-32.533333,-28.133333,2.533333,13.133333,54730.0,659300.0,172,9
7,76.266667,0.733333,5.133333,45.933333,91.266667,45.253333,49.906667,54.793333,48.760000,59.813333,...,-40.800000,16.933333,-43.666667,-40.066667,-7.666667,16.933333,51445.0,366519.0,50,2
8,86.400000,3.333333,0.800000,29.600000,86.755556,54.557778,60.308889,64.953333,60.364444,70.711111,...,-31.866667,13.533333,-33.066667,-25.933333,-3.666667,-16.733333,38243.0,413757.0,312,11
9,43.333333,0.666667,0.266667,28.133333,81.466667,50.186667,55.406667,60.786667,55.973333,68.160000,...,-36.533333,16.066667,-38.333333,-12.933333,-4.533333,-6.000000,43688.0,225763.0,108,3


In [4]:
X = df.drop(['day_21_delta_cases','day_21_delta_deaths'], axis = 1)
y = df.day_21_delta_cases

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
rf = RandomForestRegressor(max_features='auto', oob_score=True, random_state=1, n_jobs=-1)
param_grid = { "min_samples_leaf" : [1, 5, 10], "min_samples_split" : [2, 4, 8, 12], "n_estimators": [10, 50, 100, 200, 500]}
gs = GridSearchCV(n_iter = 100, estimator=rf, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
gs = gs.fit(X_train, y_train)

gs.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=4, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=-1, oob_score=True,
                      random_state=1, verbose=0, warm_start=False)

In [9]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done 162 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=4,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=500, n_jobs=-1,
                   

In [11]:
rf_random.best_params_
#print("%.4f" % rf_random.oob_score_)
print(rf_random.score(X_test, y_test))

0.4356983030409337


In [6]:
rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=4, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=-1, oob_score=True,
                      random_state=1, verbose=0, warm_start=False)

rf.fit(X_train, y_train)
print("%.4f" % rf.oob_score_)

0.7910


In [7]:
rf.score(X_test, y_test)

0.4333119523378874

In [17]:
pd.concat((pd.DataFrame(X_train.columns, columns = ['variables']), 
           pd.DataFrame(rf.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:10]


,variables,importance
0,cases,0.727740
14,mslPresMin,0.064434
16,mslPresMax,0.025178
64,pop_2018,0.022257
15,mslPresAvg,0.014539
49,windSpd80mAvg,0.010699
52,windSpd100mAvg,0.008088
59,workplaces_percent_change_from_baseline,0.007120
57,retail_and_recreation_percent_change_from_base...,0.007052
29,sfcPresAvg,0.007050


In [19]:
# death
y = df.day_21_delta_deaths
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
rf = RandomForestRegressor(max_features='auto', oob_score=True, random_state=1, n_jobs=-1)
param_grid = { "min_samples_leaf" : [1, 5, 10], "min_samples_split" : [2, 4, 8, 12], "n_estimators": [10, 50, 100, 200, 500, 1000]}
gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=-1)
gs = gs.fit(X_train, y_train)

gs.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=8, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=-1, oob_score=True,
                      random_state=1, verbose=0, warm_start=False)

In [21]:
rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=8, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=-1, oob_score=True,
                      random_state=1, verbose=0, warm_start=False)
rf.fit(X_train, y_train)
print("%.4f" % rf.oob_score_)

0.6756


In [22]:
rf.score(X_test, y_test)

0.5092299403486552

In [23]:
pd.concat((pd.DataFrame(X_train.columns, columns = ['variables']), 
           pd.DataFrame(rf.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:10]


,variables,importance
0,cases,0.774638
1,deaths,0.048380
14,mslPresMin,0.019373
18,presTendMin,0.015015
16,mslPresMax,0.012929
15,mslPresAvg,0.009576
20,presTendMax,0.008923
29,sfcPresAvg,0.007550
28,sfcPresMin,0.006895
58,residential_percent_change_from_baseline,0.006728


In [12]:
# gradient boosting

# cases
X = df.drop(['day_21_delta_cases','day_21_delta_deaths'], axis = 1)
y = df.day_21_delta_cases

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gb_hyperparameters = {
    "n_estimators": [10, 50, 100, 200, 500],
    "min_samples_split" : [2, 4, 8, 12],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.5],
    "min_samples_leaf": [1, 3, 5]
}

gbr = GradientBoostingRegressor()
gs = GridSearchCV(estimator=gbr, param_grid=gb_hyperparameters, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
gs = gs.fit(X_train, y_train)

gs.best_estimator_

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=3, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [13]:
gbr = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=3, min_samples_split=8,
                          min_weight_fraction_leaf=0.0, n_estimators=200,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)

In [14]:
print("R-squared for Train: %.2f" % gbr.score(X_train, y_train))
print("R-squared for Test: %.2f" % gbr.score(X_test, y_test))

R-squared for Train: 1.00
R-squared for Test: 0.46


In [28]:
pd.concat((pd.DataFrame(X_train.columns, columns = ['variables']), 
           pd.DataFrame(gbr.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:10]


,variables,importance
0,cases,0.713476
14,mslPresMin,0.058705
15,mslPresAvg,0.034270
30,sfcPresMax,0.032336
64,pop_2018,0.023563
16,mslPresMax,0.023362
63,income_2018,0.014859
57,retail_and_recreation_percent_change_from_base...,0.014602
59,workplaces_percent_change_from_baseline,0.013796
27,relHumMax,0.012504
